In [ ]:
!nvidia-smi

Wed Feb 24 04:34:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!wget http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz

--2021-02-24 04:34:21--  http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
Resolving www.phontron.com (www.phontron.com)... 208.113.196.149
Connecting to www.phontron.com (www.phontron.com)|208.113.196.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99246893 (95M) [application/gzip]
Saving to: ‘kftt-data-1.0.tar.gz’

kftt-data-1.0.tar.g 100%[===================>]  94.65M  37.2MB/s    in 2.5s    

2021-02-24 04:34:24 (37.2 MB/s) - ‘kftt-data-1.0.tar.gz’ saved [99246893/99246893]



In [ ]:
!tar -zxvf /content/kftt-data-1.0.tar.gz

kftt-data-1.0/
kftt-data-1.0/data/
kftt-data-1.0/data/orig/
kftt-data-1.0/data/orig/kyoto-tune.en
kftt-data-1.0/data/orig/kyoto-dev.ja
kftt-data-1.0/data/orig/kyoto-dev.en
kftt-data-1.0/data/orig/kyoto-train.en
kftt-data-1.0/data/orig/kyoto-tune.ja
kftt-data-1.0/data/orig/kyoto-train.ja
kftt-data-1.0/data/orig/kyoto-test.ja
kftt-data-1.0/data/orig/kyoto-test.en
kftt-data-1.0/data/tok/
kftt-data-1.0/data/tok/kyoto-tune.en
kftt-data-1.0/data/tok/kyoto-dev.ja
kftt-data-1.0/data/tok/kyoto-train.cln.en
kftt-data-1.0/data/tok/kyoto-dev.en
kftt-data-1.0/data/tok/kyoto-train.en
kftt-data-1.0/data/tok/kyoto-tune.ja
kftt-data-1.0/data/tok/kyoto-train.cln.ja
kftt-data-1.0/data/tok/kyoto-train.ja
kftt-data-1.0/data/tok/kyoto-test.ja
kftt-data-1.0/data/tok/kyoto-test.en
kftt-data-1.0/README.txt


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!cp -r kftt-data-1.0/data/tok/ 'drive/My Drive/'

In [ ]:
!ls '/content/drive/My Drive/tok'


kyoto-dev.en   kyoto-train.cln.en  kyoto-tune.en       limit-kyoto-test.en
kyoto-dev.ja   kyoto-train.cln.ja  kyoto-tune.ja       limit-kyoto-test.ja
kyoto-test.en  kyoto-train.en	   limit-kyoto-dev.en  limit-kyoto-train.en
kyoto-test.ja  kyoto-train.ja	   limit-kyoto-dev.ja  limit-kyoto-train.ja


In [ ]:
%cd drive/My Drive

/content/drive/My Drive


In [ ]:
!mkdir data
!mkdir save

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘save’: File exists


In [ ]:
def limiter(data_dir, src_data, tgt_data, n=50):
  '''limiter
  コーパスの最大文長を制限する。
  ----------------------------------------
  引数
  data_dir : コーパスのディレクトリ
  src_data : 原言語のデータ
  tgt_data : 目的言語のデータ
  n : 最大文長
  ----------------------------------------
  '''
  # 書き込み用のデータファイルを作成する。
  with open(f'{data_dir}/limit-{src_data}', 'w', encoding='utf8') as wsrc:
    with open(f'{data_dir}/limit-{tgt_data}', 'w', encoding='utf8') as wtgt:
      # 元のデータファイルを読み込む
      with open(f'{data_dir}/{src_data}', encoding='utf8') as rsrc:
        with open(f'{data_dir}/{tgt_data}', encoding='utf8') as rtgt:
          # それぞれ一行ずつ読み込みリストに格納する。
          src_lines = rsrc.read().strip().split('\n')
          tgt_lines = rtgt.read().strip().split('\n')
          for src, tgt in zip(src_lines, tgt_lines):
            # 最大文長がn以下のもののみ書き込む。
            if len(src.split()) <= n and len(tgt.split()) <= n:
              wsrc.write(src + '\n')
              wtgt.write(tgt + '\n')


In [ ]:
## 文長制限するファイルの一覧
files = ['train', 'dev', 'test']
for f in files:
  # 最大文長10とする
  limiter('tok', f'kyoto-{f}.en', f'kyoto-{f}.ja', n=10)



In [ ]:
!ls tok

kyoto-dev.en   kyoto-train.cln.en  kyoto-tune.en       limit-kyoto-test.en
kyoto-dev.ja   kyoto-train.cln.ja  kyoto-tune.ja       limit-kyoto-test.ja
kyoto-test.en  kyoto-train.en	   limit-kyoto-dev.en  limit-kyoto-train.en
kyoto-test.ja  kyoto-train.ja	   limit-kyoto-dev.ja  limit-kyoto-train.ja


In [ ]:
# 文長制限ができているか確認する。
!wc -l tok/limit-kyoto-train.en
!wc -l tok/limit-kyoto-train.ja

71683 tok/limit-kyoto-train.en
71683 tok/limit-kyoto-train.ja


In [ ]:
!pip install OpenNMT-py

     |████████████████████████████████| 215kB 18.4MB/s 
     |████████████████████████████████| 748.8MB 24kB/s 
     |████████████████████████████████| 2.6MB 46.0MB/s 
     |████████████████████████████████| 81kB 13.3MB/s 
     |████████████████████████████████| 61kB 11.1MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 276kB 58.9MB/s 
     |████████████████████████████████| 81kB 11.9MB/s 
     |████████████████████████████████| 1.2MB 53.4MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.3-cp37-none-any.whl size=19478 sha256=e7fa5556be67a38214d26f35e636823e9a5a66d0654008c7e1942bc472d35270
  Stored in directory: /root/.cache/pip/wheels/60/d5/5f/3001db0714a92f771c292603ef5dada52f9efa6467f3ea2bdf
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=f6f3c13772309053aca5a85006fb1a0463623184bb050997dc85824b7447a0bd
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e7

In [ ]:
# データの前処理をする。
!onmt_preprocess -train_src tok/limit-kyoto-train.en \
                 -train_tgt tok/limit-kyoto-train.ja \
                 -valid_src tok/limit-kyoto-dev.en \
                 -valid_tgt tok/limit-kyoto-dev.ja \
                 -save_data data/demo

/bin/bash: onmt_preprocess: command not found


In [ ]:
!ls data

demo.train.0.pt  demo.valid.0.pt  demo.vocab.pt


In [ ]:
!onmt_train -data data/demo \
            -save_model save \
            -layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8  \
            -encoder_type transformer -decoder_type transformer -position_encoding \
            -train_steps 10000  -max_generator_batches 2 -dropout 0.1 \
            -batch_size 1024 -batch_type tokens -normalization tokens  -accum_count 2 \
            -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 4000 -learning_rate 2 \
            -max_grad_norm 0 -param_init 0  -param_init_glorot \
            -label_smoothing 0.1 -valid_steps 2000 -save_checkpoint_steps 2000 \
            -world_size 1 -gpu_ranks 0

usage: onmt_train [-h] [-config CONFIG] [-save_config SAVE_CONFIG] -data DATA
                  [-skip_empty_level {silent,warning,error}]
                  [-transforms {sentencepiece,bpe,onmt_tokenize,filtertoolong,prefix,switchout,tokendrop,tokenmask,bart} [{sentencepiece,bpe,onmt_tokenize,filtertoolong,prefix,switchout,tokendrop,tokenmask,bart} ...]]
                  [-save_data SAVE_DATA] [-overwrite] [-n_sample N_SAMPLE]
                  [-dump_fields] [-dump_transforms] -src_vocab SRC_VOCAB
                  [-tgt_vocab TGT_VOCAB] [-share_vocab]
                  [-src_vocab_size SRC_VOCAB_SIZE]
                  [-tgt_vocab_size TGT_VOCAB_SIZE]
                  [-vocab_size_multiple VOCAB_SIZE_MULTIPLE]
                  [-src_words_min_frequency SRC_WORDS_MIN_FREQUENCY]
                  [-tgt_words_min_frequency TGT_WORDS_MIN_FREQUENCY]
                  [--src_seq_length_trunc SRC_SEQ_LENGTH_TRUNC]
                  [--tgt_seq_length_trunc TGT_SEQ_LENGTH_TRUNC]
          

In [ ]:
!onmt_translate -model save/_step_2000.pt \
                -src tok/limit-kyoto-test.en \
                -output tok/limit-kyoto-output.ja \
                -replace_unk -verbose

Traceback (most recent call last):
  File "/usr/local/bin/onmt_translate", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/translate.py", line 44, in main
    translate(opt)
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/translate.py", line 15, in translate
    translator = build_translator(opt, logger=logger, report_score=True)
  File "/usr/local/lib/python3.7/dist-packages/onmt/translate/translator.py", line 32, in build_translator
    fields, model, model_opt = load_test_model(opt)
  File "/usr/local/lib/python3.7/dist-packages/onmt/model_builder.py", line 85, in load_test_model
    map_location=lambda storage, loc: storage)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 571, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 229, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/li